<a href="https://colab.research.google.com/github/robgon-art/InventorBot/blob/main/InventorBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **InventorBot: Using AI to Generate Ideas for Inventions in Any Field**
How a neural network trained on the US Patent Database can produce new and possibly useful inventions.

![alt text](https://raw.githubusercontent.com/robgon-art/InventorBot/main/Shadow_Hand_Bulb_small.png)

# Initialize the InventorBot System
Click the **Run cell** button (the play button) below to set things up. It takes about 5 minutes to load the dataset files and configure the neural network models.

In [ ]:
#@title Initialize System
!pip install -q gpt-2-simple
print("Downloading the InventorBot model.")
!gdown 1GReZO3uUflXP8rYhdFjfrw5OJlEXlZoR
!unzip inventor_bot.zip
!pip install transformers==3.4.0
from transformers import pipeline
summarizer = pipeline("summarization", model="t5-large", tokenizer="t5-large", framework="pt")
import gpt_2_simple as gpt2
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)
import textwrap
wrapper = textwrap.TextWrapper(width=175) 
import os
import sys
old_stderr = sys.stderr
f = open(os.devnull, 'w')
sys.stderr = f
setup_gruen = False

# Create Some New Ideas for Inventions
Type in some **keywords** then click **Run cell** button to generate new invention ideas with titles and abstracts that relate to the keywords. You can run this multiple times with the same or different keywords. You can specify the **temperature** to change the quality and diversity of the text. Higher temperatures will have greater linguistic variety where lower temperatures will be more grammatically correct. You can also specify the **number of ideas** to generate.

In [2]:
keywords = "improved headphones" #@param {type:"string"}
temperature = 85 #@param {type:"slider", min:70, max:100, step:1}
number_of_ideas = 7 #@param {type:"slider", min:4, max:20, step:1}

new_ideas = gpt2.generate(sess, length=200, temperature=temperature/100,
  prefix="KEYWORDS: " + keywords + "\nTITLE:", nsamples=number_of_ideas, 
  batch_size=number_of_ideas, return_as_list=True, include_prefix=True,
  truncate="***")

keepers = []
summaries = []

show_full_abstract = False

for idea in new_ideas:
  if idea.count("KEYWORDS:") is 1 and idea.count("TITLE:") is 1 and idea.count("ABSTRACT:") is 1:
    parts = idea.split("\n")
    if len(parts) is not 4:
      continue
    keeper = []
    for part in parts:
      if part.startswith("ABSTRACT:"):
        abstract = part[9:]
        if show_full_abstract:
          part = "FULL ABSTRACT: " + abstract + "\n"
        else:
          part = ""
        summary_text = summarizer(abstract, max_length=200, min_length=5, do_sample=False)[0]['summary_text']
        summary_text = summary_text + " "
        summary_text ='. '.join([i.capitalize() for i in summary_text.split(' . ')])
        if "null pointer" in summary_text:
          summaries.append(abstract)
          part += "ABSTRACT: " + abstract
        else:
          summaries.append(summary_text)
          part += "ABSTRACT: " + summary_text
      if len(part) > 0:
        keeper.append(part)
      print(wrapper.fill(part))
    keepers.append(keeper)

KEYWORDS: improved headphones
TITLE: Podcast distribution system consisting of improved headphones
ABSTRACT: A podcast distribution system is provided that comprises a first antenna and a second antenna coupled via the first antenna to form a mobile station. First receiver
has a digital radio receiver coupled to the radio system.

KEYWORDS: improved headphones
TITLE: Method and apparatus for improved headphones
ABSTRACT: Improved speaker system includes a recessed sound stream. Speaker recirculating apparatus recirculates sound stream into speaker.

KEYWORDS: improved headphones
TITLE: Threshold ratio of improved headphones
ABSTRACT: Improved headphones system includes an acoustic driver, an earphone, a speaker, and an amplifier. A pair of semiconducting leads are connected to a base base-mount.

KEYWORDS: improved headphones
TITLE: Removable earphone having improved headphones
ABSTRACT: Removably binding earphone includes a housing having an earpiece. Plurality of headphones are conne